安装包
- pip install tushare
- pip install arch

In [5]:
import pandas as pd
import numpy as np
from scipy.stats import norm


In [6]:
from arch import arch_model

In [2]:
import tushare as ts
ts.set_token('**你的KEY*')
pro = ts.pro_api()

In [9]:
data = pro.query('stock_basic', exchange='', list_status='L', fields='ts_code,symbol,name,area,industry,list_date')

In [10]:
data.head()

,ts_code,symbol,name,area,industry,list_date
0,000001.SZ,000001,平安银行,深圳,银行,19910403
1,000002.SZ,000002,万科A,深圳,全国地产,19910129
2,000004.SZ,000004,*ST国华,深圳,软件服务,19910114
3,000006.SZ,000006,深振业A,深圳,区域地产,19920427
4,000007.SZ,000007,全新好,深圳,其他商业,19920413


In [19]:
ticker_ts = "600570.SH" 
data = pro.daily(ts_code=ticker_ts, 
                             start_date='20230101', 
                             end_date='20230530',
                             fields='trade_date, close, pre_close')

In [14]:
data.head()

,trade_date,close,pre_close
0,20230530,42.30,42.34
1,20230529,42.34,42.13
2,20230526,42.13,41.58
3,20230525,41.58,42.06
4,20230524,42.06,42.50


In [22]:
from sklearn.linear_model import Ridge # 使用 Ridge 回归进行时间序列预测模拟
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [20]:
def conduct_ml_analysis(asset_type: str, risk_level: str,data) -> str:
    """
    调用金融机器学习模型，分析特定资产在特定风险下的潜在表现。
    输入 asset_type 必须是 '股票', '债券', '基金' 中的一种。
    输入 risk_level 必须是 '保守', '稳健', '激进' 中的一种。
    返回分析报告的摘要。
    """
    
    if risk_level == '激进' and asset_type == '股票':
        ticker_ts = "600570.SH" # 恒生电子，高成长性 A 股模拟

        try:
            # 1. 获取历史数据
            #data = pro.daily(ts_code=ticker_ts, 
            #                 start_date='20220101', 
            #                 end_date='20220530',
            #                 fields='trade_date, close, pre_close')
            
            if data.empty:
                 return f"ML模型分析失败：无法获取股票 {ticker_ts} 的历史数据。"

            # 数据处理：计算对数收益率
            data['Log_Return'] = np.log(data['close'] / data['pre_close'])
            data = data.sort_values(by='trade_date')
            data = data.dropna().reset_index(drop=True)
            
            # --- 2. 收益率预测 (使用 Ridge 回归模拟) ---
            # 创建特征 (Lagged features)：使用前 5 天的收益率、波动率作为特征
            N = 5
            for i in range(1, N + 1):
                data[f'Lag_Return_{i}'] = data['Log_Return'].shift(i)
            data['MA_Return'] = data['Log_Return'].rolling(window=N).mean().shift(1)
            data['Volatility'] = data['Log_Return'].rolling(window=N).std().shift(1)
            
            data = data.dropna()
            
            # 目标：下一天的收益率 (Log_Return)
            X = data.drop(columns=['trade_date', 'close', 'pre_close', 'Log_Return'])
            y = data['Log_Return']
            
            # 训练集和预测特征 (取最后一行作为预测输入)
            X_train = X.iloc[:-1]
            y_train = y.iloc[:-1]
            X_pred = X.iloc[-1].values.reshape(1, -1)
            
            # 标准化特征
            scaler = StandardScaler()
            X_train_scaled = scaler.fit_transform(X_train)
            X_pred_scaled = scaler.transform(X_pred)
            
            # 训练 Ridge 模型
            ridge_model = Ridge(alpha=1.0) # alpha 是正则化强度
            ridge_model.fit(X_train_scaled, y_train)
            
            # 预测下一日的对数收益率
            next_day_log_return = ridge_model.predict(X_pred_scaled)[0]
            
            # 模拟计算年化收益率
            annual_return_forecast = (np.exp(next_day_log_return * 250) - 1) * 100
            
            # --- 3. 风险预测 (使用 GARCH 模型模拟波动率) ---
            
            # 拟合 GARCH(1,1) 模型来预测波动率
            am = arch_model(100 * data['Log_Return'], vol='Garch', p=1, q=1)
            res = am.fit(update_freq=5, disp='off')
            
            # 获取下一日的条件波动率 (年化)
            forecasts = res.forecast(horizon=1)
            next_day_variance = forecasts.variance.iloc[-1, 0] / 10000 # 转换为小数
            
            annual_volatility_forecast = np.sqrt(next_day_variance * 250)
            
            # 风险评估：最大回撤 VaR 模拟
            # 使用预测的波动率来计算 99% VaR
            z_score_99 = norm.ppf(0.01) # 约 -2.33
            VaR_99_daily = z_score_99 * np.sqrt(next_day_variance)
            max_drawdown_sim = -VaR_99_daily * 100 * 5 # 放大倍数模拟极端回撤

            # --- 4. 组装分析报告 ---
            
            result = (f"**【高成长 A 股 ML 分析报告】**\n"
                      f"资产代码：{ticker_ts} (A股模拟)\n"
                      f"**ML 预测年化收益率 (Ridge)：** 约 {annual_return_forecast:.2f}%\n"
                      f"**预测年化波动率 (GARCH)：** 约 {annual_volatility_forecast:.2f}%\n"
                      f"**风险评估 (99% VaR 模拟)：** 约 {max_drawdown_sim:.2f}% (指在99%的置信水平下，最大潜在损失的简化估计)。\n"
                      f"**综合建议：** 机器学习模型预测短期存在高波动性，但长期收益潜力大。此配置仅适合资本损失承受能力强的激进型投资者，建议严格执行止损策略。"
                     )
            return result
            
        except Exception as e:
            return f"ML模型分析工具遇到错误，无法完成预测和风险评估：请检查Tushare Token/网络/数据权限，原始错误：{e}"

    # --- 其他风险场景保持不变 ---
    elif risk_level == '保守' and asset_type == '债券':
        return "模型分析：高等级企业债券在当前市场环境下流动性较好，预计年化收益在 4%-6% 之间，风险极低，适合保值。"
    # ... (其他分支不变)
    else:
        return f"模型对 {risk_level} 投资者配置 {asset_type} 的分析结果暂无高置信度报告，建议结合RAG知识库中的原则进行配置。"

In [23]:
conduct_ml_analysis('股票','激进',data)

11


/Users/billsun/miniforge3/envs/rag/lib/python3.10/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


'**【高成长 A 股 ML 分析报告】**\n资产代码：600570.SH (A股模拟)\n**ML 预测年化收益率 (Ridge)：** 约 63.14%\n**预测年化波动率 (GARCH)：** 约 0.28%\n**风险评估 (99% VaR 模拟)：** 约 20.41% (指在99%的置信水平下，最大潜在损失的简化估计)。\n**综合建议：** 机器学习模型预测短期存在高波动性，但长期收益潜力大。此配置仅适合资本损失承受能力强的激进型投资者，建议严格执行止损策略。'